Purpose: Turn text-data into data with appropriate values neccessary for functional RAGAS evaluation

Fetch documents (from only the SciComp wiki as of now) and put into accessable format for generation of expected outputs and creation of context later on

In [3]:
# from langchain_community.document_loaders import DirectoryLoader
# # import nltk
# # nltk.download('punkt')
# loader = DirectoryLoader("../data/test")
# documents = loader.load()


from langchain_community.document_loaders import TextLoader
import os
import pandas as pd


loader = TextLoader("../data/test/test.txt")
documents = loader.load()
print(documents)



# #Code that indexes over all the documents in the Data folder below 
# import glob
# import os

# class TextLoader:
#     def __init__(self, base_path):
#         self.base_path = base_path

#     def load(self):
#         documents = []
#         # Use glob.glob with recursive=True to find all .txt files within the base_path directory and its subdirectories
#         #refactor to work with non txt files
#         for file_path in glob.glob(os.path.join(self.base_path, '**/*.txt'), recursive=True):
#             with open(file_path, 'r', encoding='utf-8') as file:
#                 documents.append(file.read())
#         return documents

# # Assuming the base_path is set correctly relative to the notebook's location
# loader = TextLoader("../data")
# documents = loader.load()
# print(documents)

[Document(page_content='Overview of Janelia Scientific Computing\nThe Janelia Scientific Computing team supports the institute’s mission by developing and maintaining computational infrastructure, software tools, and data analysis pipelines essential for modern biological research. They collaborate closely with scientists to address computational challenges and drive innovation in various research areas, including neuroscience, cell biology, and bioinformatics.\n\nKey Areas of Focus\nHigh-Performance Computing (HPC):\n\nProvides access to powerful computing clusters to handle large-scale data analysis and complex simulations.\nOptimizes computational workflows to maximize efficiency and resource utilization.\nData Management and Storage:\n\nDevelops and maintains robust data storage solutions to handle the vast amounts of data generated by research activities.\nEnsures data integrity, security, and accessibility.\nSoftware Development:\n\nCreates custom software tools and platforms tai

Generate the testset

In [4]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
import os
# generator with openai models
from dotenv import load_dotenv
import os

load_dotenv()  # Load .env file
api_key = os.environ.get("OPENAI_API_KEY")

#change the model to a better one once the whole thing is functional

generator_llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=api_key)
# critic_llm = ChatOpenAI(model="gpt-4")
critic_llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=api_key)
embeddings = OpenAIEmbeddings(api_key=api_key)

generator = TestsetGenerator.from_langchain(
    generator_llm,
    critic_llm,
    embeddings
) 


testset = generator.generate_with_langchain_docs(documents, test_size=10, distributions={simple: 0.5, reasoning: 0.25, multi_context: 0.25})

embedding nodes:   0%|          | 0/2 [00:00<?, ?it/s]

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function AsyncEmbeddings.create.<locals>.parser at 0x3090367a0>, 'json_data': {'input': [[42144, 315, 622, 2444, 689, 38130, 46879, 198, 791, 622, 2444, 689, 38130, 46879, 2128, 11815, 279, 44798, 753, 9131, 555, 11469, 323, 20958, 55580, 14054, 11, 3241, 7526, 11, 323, 828, 6492, 58773, 7718, 369, 6617, 24156, 3495, 13, 2435, 51696, 15499, 449, 14248, 311, 2686, 55580, 11774, 323, 6678, 19297, 304, 5370, 3495, 5789, 11, 2737, 93048, 11, 2849, 34458, 11, 323, 17332, 98588, 382, 1622, 56816, 315, 26891, 198, 12243, 12, 35891, 46879, 320, 39, 4977, 7887, 50122, 2680, 311, 8147, 25213, 28066, 311, 3790, 3544, 13230, 828, 6492, 323, 6485, 47590, 627, 22078, 318, 4861, 55580, 88568, 311, 35608, 15374, 323, 5211, 50549, 627, 1061, 9744, 323, 15035, 1473, 21076, 82, 323, 33095, 22514, 828, 5942, 10105, 311, 3790, 279, 13057, 15055, 315, 828, 8066, 555, 3495, 7640, 627, 97296, 143

embedding nodes:  50%|█████     | 1/2 [00:00<00:00,  5.02it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
DEBUG:openai._base_client:HTTP Request: POST https://api.openai.com/v1/chat/completions "200 OK"


INFO:ragas.testset.docstore:Document [ID: b1f04e59-dd31-4ad8-8934-9a670ebe1f51] has no filename, using `doc_id` instead
INFO:ragas.testset.docstore:Document [ID: b1f04e59-dd31-4ad8-8934-9a670ebe1f51] has no filename, using `doc_id` instead


...

Generating:  80%|████████  | 8/10 [00:25<00:09,  4.92s/it]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
DEBUG:openai._base_client:HTTP Request: POST https://api.openai.com/v1/chat/completions "200 OK"
INFO:ragas.testset.evolutions:retrying evolution: 3 times
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'content': '\n    Given a context, perform the following task and output the answer in VALID JSON format: Assess the provided context and assign a numerical score of 1 (Low), 2 (Medium), or 3 (High) for each of the following criteria in your JSON response:\n\nclarity: Evaluate the precision and understandability of the information presented. High scores (3) are reserved for contexts that are both precise in their information and easy to understand. Low scores (1) are for contexts where the information is vague or hard to comprehend.\ndepth: Determine the level of detailed examination and the inclusion of innovativ

Generating:  90%|█████████ | 9/10 [00:29<00:04,  4.70s/it]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
DEBUG:openai._base_client:HTTP Request: POST https://api.openai.com/v1/chat/completions "200 OK"
DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'content': 'Generate a question that can be fully answered from given context. The question should be formed using topic\n\nExamples:\n\ncontext: "Photosynthesis in plants involves converting light energy into chemical energy, using chlorophyll and other pigments to absorb light. This process is crucial for plant growth and the production of oxygen."\nkeyphrase: "Photosynthesis"\nquestion: "What is the role of photosynthesis in plant growth?"\n\ncontext: "The Industrial Revolution, starting in the 18th century, marked a major turning point in history as it led to the development of factories and urbanization."\nkeyphrase: "Industrial Revolution"\nquestion: "How did the I

Generating: 100%|██████████| 10/10 [00:52<00:00,  5.23s/it]


Address potentiall duplicate questions (which the testset generator has done) and delete their rows

In [6]:
df = testset.to_pandas()
df = df.drop_duplicates(subset='question', keep='first')
questions_list = df['question'].tolist()
print (questions_list)

seen = set()
questions_list = [x for x in questions_list if not (x in seen or seen.add(x))]

for item in questions_list:
    print(item)

    

# testset.to_json("testset.json")
# Creates dataset of ground truths contexts and questions for the testset
# Missing answers column 
# On one medium size document, it took about 1 minutes to generate 10 questions and cost 3 dollars on OpenAI
# Seems expensive when using gpt-4, is its use justified or does 3.5 get the job done?
# Evaluate gpt-4 vs gpt-3.5-turbo-16k for RAGAS evaluation test data generation


["How does the Janelia Scientific Computing team support the institute's mission and drive innovation in modern biological research?", 'How does the Janelia Scientific Computing team utilize high-performance computing to handle large-scale data analysis and complex simulations in modern biological research?', 'How does the Janelia Scientific Computing team utilize bioinformatics to analyze genomic, proteomic, and other biological data?', 'What are some notable projects and contributions of the Janelia Scientific Computing team?', "How does the Janelia Scientific Computing team support the institute's mission and promote innovation in biological research through collaborations and addressing computational challenges in neuroscience, cell biology, and bioinformatics?", 'What projects by the Janelia Scientific Computing team have impacted neuroscience, cell biology, and bioinformatics?', 'How does the Janelia team use Neuroglancer to enhance data visualization in neuroscience research?']


In [7]:
df.to_csv('testset.csv', index=False)

fetch the LLM's response and append to a list

In [8]:
# Assuming SemanticSearchParser is your class and it has a method named 'generate_response' that takes a question and returns an answer

# Initialize your SemanticSearchParser class
# Adjust this step if your class initialization requires different parameters


# Now you can import the class

from generate_answer import SemanticSearchService

weaviate_url = "http://localhost:8777"
service = SemanticSearchService(weaviate_url)
print (questions_list)

# List to store answers (optional)
answers_list = []


# Loop through each question in the questions_list
for question in questions_list:
    # Use the question as input to get the answer
    answer = service.generate_response(question)
    
    # Print the answer
    
    # Optionally, append the answer to answers_list for further processing
    answers_list.append(answer)

print (answers_list)
temp_ans_list = ["The Janelia Scientific Computing team operates and maintains a world-class computational infrastructure that includes a high-performance compute cluster with over 5000 cores and 300 GPUs. This infrastructure is used to analyze and mine the large amounts of data produced by Janelia's scientists. The team also supports a state-of-the-art storage and compute infrastructure across two data centers, which currently supports over 15 petabytes of scientific data. This data is split across various storage tiers and is connected with an optical fiber ring. The team also maintains a 4500 sq ft data center with significant power and cooling capacity. \n\nIn addition to hardware, the team also has deep software skills in a broad range of programming languages, extendable applications, frameworks, cloud & cluster technologies, and databases. These skills are used to help with research and engineering tasks, from quick questions to full software life cycle support. The team's software skills, combined with their domain knowledge in areas such as image processing, machine learning, data handling, microscopy, instrument control, 3D graphics & visualization, and bioinformatics & transcriptomics, allow them to efficiently work with both experimentalists and computer scientists. \n\nThe team also develops and maintains a variety of tools and projects, such as NeuronBridge, HortaCloud, VVD Viewer, EASI-FISH pipeline, Render, RS-FISH, and BigStitcher, which are used for various aspects of data analysis and simulation in biological research.", "The Janelia Scientific Computing team provides a wide range of support for advanced imaging techniques and image analysis. They offer consultation on experiment design as well as image visualization and processing. They also provide comprehensive image and data analysis support for multiple software packages through hands-on assistance and/or custom-written macros/plugins/scripts for ImageJ/FIJI, MATLAB, Imaris, etc. \n\nIn addition, they maintain several computer workstations dedicated to viewing and processing large image datasets acquired with the facility's instruments. These workstations are equipped with a suite of imaging software, including a full version of Imaris, and have robust hardware specifications to handle large datasets. \n\nThe team also has deep domain knowledge in image processing, machine learning, data handling, and 3D graphics & visualization, which allows them to efficiently work with experimentalists and computer scientists in various research areas.", "The Janelia Scientific Computing team provides world-class computational infrastructure to support the institute's scientific endeavors. They operate and maintain all of Janelia’s storage and associated backup infrastructure, high performance compute cluster, and all Linux systems. They also manage Janelia’s data center and backup and disaster recovery resources. The team supports a Linux compute cluster with over 5000 cores and 300 GPUs, and is responsible for maintaining many other Linux servers and workstations. They also handle a significant amount of data, with almost 100TB of new Janelia’s data being safely backed up every month.\n\nIn addition to infrastructure, the Scientific Computing Software team works closely with Janelia's labs and project teams, providing everything from answering quick questions to full software life cycle support. They have a broad range of software skills, including programming languages, extendable applications, frameworks, cloud & cluster technologies, and databases. They also have deep domain knowledge in areas like image processing, machine learning, data handling, microscopy, instrument control, 3D graphics & visualization, and bioinformatics & transcriptomics. \n\nThe team also identifies opportunities for code reuse, reducing development overhead and support costs across Janelia. They are strong proponents of open science and have created the Open Science Software Initiative. Most of their software is open source and available via GitHub. They also run a Scientific Computing Associates program to embed associates in SciComp and the lab or team they work with. \n\nThe team is led by Stephan Preibisch and consists of three teams: Software Engineering headed by Konrad Rokicki, Computational Methods and Solutions, both headed by Stephan Preibisch. They have developed several tools and projects like NeuronBridge, HortaCloud, VVD Viewer, EASI-FISH pipeline, Render, RS-FISH, and BigStitcher. \n\nIn summary, the Janelia Scientific Computing team supports the institute's mission and drives innovation in modern biological research by providing robust computational infrastructure, software support, and developing innovative tools and solutions.", 'The Janelia Scientific Computing team supports advanced imaging techniques in neuroscience and cell biology through a variety of ways. They have deep domain knowledge in image processing, machine learning, data handling, electron and light microscopy, instrument control, 3D graphics & visualization, bioinformatics & transcriptomics. They also develop and maintain a range of software tools and applications that aid in these areas. Some of these tools include NeuronBridge for finding neuron matches across modalities, HortaCloud for cloud-based collaborative annotation, VVD Viewer for volumetric rendering of 3D/4D microscopy data, and BigStitcher for efficient alignment of multi-tile and multi-angle image datasets. They also work closely with labs and project teams, providing full software life cycle support.', "The Janelia Scientific Computing team supports modern biological research in several ways. They maintain a world-class computational infrastructure, including storage and backup infrastructure, a high-performance compute cluster, and all Linux systems. They also manage Janelia's data center and backup and disaster recovery resources. The team supports data storage infrastructure for storing and accessing scientific data, with over 15 petabytes of scientific data split across various storage tiers. They also support a Linux compute cluster with over 5000 cores and 300 GPUs, and maintain many other Linux servers and workstations. \n\nIn addition to infrastructure support, the Scientific Computing Software team works closely with Janelia's labs, project teams, and shared resources to help with research and engineering tasks. They provide everything from answering quick questions to full software life cycle support. The team's software skills span a broad range of programming languages, extendable applications, frameworks, cloud & cluster technologies, and databases. They also have deep domain knowledge in image processing, machine learning, data handling, electron and light microscopy, instrument control, 3D graphics & visualization, bioinformatics & transcriptomics. \n\nThe team also develops and maintains a variety of tools and projects, such as NeuronBridge, HortaCloud, VVD Viewer, EASI-FISH pipeline, Render, RS-FISH, and BigStitcher. They are strong proponents of open science and most of their software is open source and available via GitHub. They also run the Scientific Computing Associates program, which offers challenging assignments for those interested in computational science.", "The Janelia Scientific Computing team provides comprehensive support for advanced imaging techniques in neuroscience, cell biology, and bioinformatics through a variety of collaborations and custom software tools. They work closely with Janelia's labs, project teams, and shared resources to assist with research and engineering tasks. This can range from answering quick questions to providing full software life cycle support.\n\nThe team's software skills cover a broad range of programming languages, extendable applications, frameworks, cloud & cluster technologies, and databases. They have deep domain knowledge in image processing, machine learning, data handling, electron and light microscopy, instrument control, 3D graphics & visualization, bioinformatics & transcriptomics. Many team members have backgrounds in biology, enabling them to work efficiently with experimentalists and computer scientists.\n\nThe team is also involved in various projects and tools such as NeuronBridge, HortaCloud, VVD Viewer, EASI-FISH pipeline, Render, RS-FISH, and BigStitcher, which are designed to support advanced imaging techniques and data analysis in neuroscience, cell biology, and bioinformatics.\n\nFurthermore, the team is a strong proponent of open science and has teamed up with the Computation & Theory research area to create the Open Science Software Initiative. Most of their software is open source and available via GitHub, promoting collaboration and knowledge sharing. They also run the Scientific Computing Associates program, which embeds associates in SciComp and the lab or team they work with."]




["How does the Janelia Scientific Computing team support the institute's mission and drive innovation in modern biological research?", 'How does the Janelia Scientific Computing team utilize high-performance computing to handle large-scale data analysis and complex simulations in modern biological research?', 'How does the Janelia Scientific Computing team utilize bioinformatics to analyze genomic, proteomic, and other biological data?', 'What are some notable projects and contributions of the Janelia Scientific Computing team?', "How does the Janelia Scientific Computing team support the institute's mission and promote innovation in biological research through collaborations and addressing computational challenges in neuroscience, cell biology, and bioinformatics?", 'What projects by the Janelia Scientific Computing team have impacted neuroscience, cell biology, and bioinformatics?', 'How does the Janelia team use Neuroglancer to enhance data visualization in neuroscience research?']


Add values from the list of JaneliaGPT responses to the dataframe

In [9]:
df['answer'] = None
# Assuming df is your DataFrame and answers_list is a list with values to populate the 'Answer' column
if len(df) == len(answers_list):
    df['answer'] = answers_list
else:
    print("The length of answers_list does not match the number of rows in the DataFrame.")

df.dropna(axis=1, how='all', inplace=True)


In [10]:


display(df)
df.to_json('WithAnswersDatasetFromTestTxt.json', index=True)

,question,contexts,ground_truth,evolution_type,metadata,episode_done,answer
0,How does the Janelia Scientific Computing team...,[Overview of Janelia Scientific Computing\nThe...,The Janelia Scientific Computing team supports...,simple,[{'source': '../data/test/test.txt'}],True,The Janelia Scientific Computing team supports...
1,How does the Janelia Scientific Computing team...,[Overview of Janelia Scientific Computing\nThe...,The Janelia Scientific Computing team utilizes...,simple,[{'source': '../data/test/test.txt'}],True,The Janelia Scientific Computing team utilizes...
2,How does the Janelia Scientific Computing team...,[Overview of Janelia Scientific Computing\nThe...,The Janelia Scientific Computing team utilizes...,simple,[{'source': '../data/test/test.txt'}],True,The Janelia Scientific Computing team utilizes...
3,What are some notable projects and contributio...,[Overview of Janelia Scientific Computing\nThe...,Some notable projects and contributions of the...,simple,[{'source': '../data/test/test.txt'}],True,"NeuronBridge, HortaCloud, VVD Viewer, EASI-FIS..."
5,How does the Janelia Scientific Computing team...,[Overview of Janelia Scientific Computing\nThe...,The Janelia Scientific Computing team supports...,reasoning,[{'source': '../data/test/test.txt'}],True,The Janelia Scientific Computing team supports...
6,What projects by the Janelia Scientific Comput...,[Overview of Janelia Scientific Computing\nThe...,The Janelia Scientific Computing team has impa...,reasoning,[{'source': '../data/test/test.txt'}],True,"NeuronBridge, HortaCloud, VVD Viewer, EASI-FIS..."
7,How does the Janelia team use Neuroglancer to ...,[Overview of Janelia Scientific Computing\nThe...,Neuroglancer is an open-source web-based tool ...,reasoning,[{'source': '../data/test/test.txt'}],True,The Janelia team uses Neuroglancer to enhance ...


In [11]:
#Attempt to fix the issue with the answers_list not bsjdkflasdjjfklasdkl fasdjkfh jkas
df_fix = df
# List of columns to keep
columns_to_keep = ['question', 'ground_truth', 'answer', 'contexts']

# Reassign df to a DataFrame containing only the columns to keep
df_fix= df_fix[columns_to_keep]

display(df_fix['contexts'])

0    [Overview of Janelia Scientific Computing\nThe...
1    [Overview of Janelia Scientific Computing\nThe...
2    [Overview of Janelia Scientific Computing\nThe...
3    [Overview of Janelia Scientific Computing\nThe...
5    [Overview of Janelia Scientific Computing\nThe...
6    [Overview of Janelia Scientific Computing\nThe...
7    [Overview of Janelia Scientific Computing\nThe...
Name: contexts, dtype: object

In [12]:
# Assuming df is already defined and contains the necessary columns

# Convert 'question' and 'answer' to lists of strings if they are not already
df_fix['question'] = df_fix['question'].apply(lambda x: [x] if isinstance(x, str) else x)
df_fix['answer'] = df_fix['answer'].apply(lambda x: [x] if isinstance(x, str) else x)

# Ensure 'contexts' and 'ground_truth' are lists of lists of strings
# This step assumes 'contexts' and 'ground_truth' are already in the correct format
# If not, you would need to apply a similar conversion as above, ensuring each element is a list

# Example conversion if 'contexts' and 'ground_truth' were not already lists of lists
df_fix['contexts'] = df_fix['contexts'].apply(lambda x: [[y] for y in x] if isinstance(x, list) and all(isinstance(y, str) for y in x) else x)
df_fix['ground_truth'] = df_fix['ground_truth'].apply(lambda x: [[y] for y in x] if isinstance(x, list) and all(isinstance(y, str) for y in x) else x)

# Now df should be in the correct format for training

In [13]:
#  This converts to dictionary, dont think we need this right now

# # Select the first row of the DataFrame (assuming it's the one you want to convert)
# row = df_fix.iloc[0]

# # Convert the selected row to a dictionary
# result_dict = {
#     'question': row['question'][0] if row['question'] else None,  # Assuming 'question' is a list of strings
#     'answer': row['answer'][0] if row['answer'] else None,  # Assuming 'answer' is a list of strings
#     'contexts': row['contexts'] if row['contexts'] else [],  # Assuming 'contexts' is already a list of strings
#     'ground_truths': row['ground_truth'] if row['ground_truth'] else []  # Renaming key and assuming it's a list of lists
# }
# print (result_dict)
# # result_dict now resembles the desired format

# formatted_dataset = {
#     'features': list(result_dict.keys()),  # List of keys from result_dict
#     'num_rows': 1  # Since we're only dealing with a single row
# }

# print(formatted_dataset)

{'question': "How does the Janelia Scientific Computing team support the institute's mission and drive innovation in modern biological research?", 'answer': "The Janelia Scientific Computing team supports the institute's mission and drives innovation in modern biological research by providing world-class computational infrastructure and software support. They maintain a high-performance compute cluster, storage infrastructure, and data center capable of handling large amounts of scientific data. Additionally, the team collaborates closely with Janelia's labs and project teams, offering a wide range of software skills and domain knowledge in areas such as image processing, machine learning, and data handling. Their focus on open science, code reuse, and collaboration with the scientific community helps facilitate cutting-edge research and innovation in biological sciences at Janelia.", 'contexts': [['Overview of Janelia Scientific Computing\nThe Janelia Scientific Computing team support

In [23]:
from datasets import load_dataset
import os

amnesty_qa = load_dataset("explodinggradients/amnesty_qa", "english_v2")
display (amnesty_qa["eval"])
display(df_fix)
#  print (formatted_dataset)

Repo card metadata block was not found. Setting CardData to empty.


Dataset({
    features: ['question', 'ground_truth', 'answer', 'contexts'],
    num_rows: 20
})

,question,ground_truth,answer,contexts
0,[How does the Janelia Scientific Computing tea...,The Janelia Scientific Computing team supports...,[The Janelia Scientific Computing team support...,[[Overview of Janelia Scientific Computing\nTh...
1,[How does the Janelia Scientific Computing tea...,The Janelia Scientific Computing team utilizes...,[The Janelia Scientific Computing team utilize...,[[Overview of Janelia Scientific Computing\nTh...
2,[How does the Janelia Scientific Computing tea...,The Janelia Scientific Computing team utilizes...,[The Janelia Scientific Computing team utilize...,[[Overview of Janelia Scientific Computing\nTh...
3,[What are some notable projects and contributi...,Some notable projects and contributions of the...,"[NeuronBridge, HortaCloud, VVD Viewer, EASI-FI...",[[Overview of Janelia Scientific Computing\nTh...
5,[How does the Janelia Scientific Computing tea...,The Janelia Scientific Computing team supports...,[The Janelia Scientific Computing team support...,[[Overview of Janelia Scientific Computing\nTh...
6,[What projects by the Janelia Scientific Compu...,The Janelia Scientific Computing team has impa...,"[NeuronBridge, HortaCloud, VVD Viewer, EASI-FI...",[[Overview of Janelia Scientific Computing\nTh...
7,[How does the Janelia team use Neuroglancer to...,Neuroglancer is an open-source web-based tool ...,[The Janelia team uses Neuroglancer to enhance...,[[Overview of Janelia Scientific Computing\nTh...


Run the evaluation given the dataframe (does not work)

In [28]:
from datasets import Dataset
dataset_fix = Dataset.from_pandas(df_fix)
dataset_fix = dataset_fix.remove_columns(['__index_level_0__'])

print (dataset_fix)
print (amnesty_qa["eval"])

Dataset({
    features: ['question', 'ground_truth', 'answer', 'contexts'],
    num_rows: 7
})
Dataset({
    features: ['question', 'ground_truth', 'answer', 'contexts'],
    num_rows: 20
})


In [40]:
from datasets import Features

# Assuming dataset_fix and amnesty_qa["eval"] are your datasets

# Access the Features of each dataset
features_dataset_fix = dataset_fix.features
features_amnesty_eval = amnesty_qa["eval"].features
def format_columns(example):
    # Format 'question', 'answer', and 'ground_truths' columns to single values
    for column in ['question', 'answer', 'ground_truths']:
        if column in example and example[column]:
            example[column] = example[column][0]
    
    # Correctly format 'contexts' column to a list of list of strings
    if 'contexts' in example:
        # Ensure 'contexts' is a list of strings (not a list of lists)
        if isinstance(example['contexts'], list):
            # If the items are lists (or other non-string types), flatten and convert to strings
            example['contexts'] = [str(item) for sublist in example['contexts'] for item in (sublist if isinstance(sublist, list) else [sublist])]
        else:
            # If 'contexts' is not a list, convert it into a list of a single string
            example['contexts'] = [str(example['contexts'])]

    
    return example


# Apply the transformation to both datasets
dataset_fix = dataset_fix.map(format_columns)
# Direct comparison of data types
if set(features_dataset_fix.keys()) == set(features_amnesty_eval.keys()):
    all_types_match = True
    for key in features_dataset_fix.keys():
        type_dataset_fix = type(features_dataset_fix[key]).__name__
        type_amnesty_eval = type(features_amnesty_eval[key]).__name__
        if type_dataset_fix != type_amnesty_eval:
            print(f"Data type for feature '{key}' differs between datasets. dataset_fix: {type_dataset_fix}, amnesty_qa['eval']: {type_amnesty_eval}")
            all_types_match = False
    if all_types_match:
        print("The data types of all features in both datasets match.")
else:
    print("The Features of the datasets differ in their keys.")

Map: 100%|██████████| 7/7 [00:00<00:00, 1818.64 examples/s]

The data types of all features in both datasets match.


In [41]:



from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
)


from ragas import evaluate


# Will reutrn a dataframe with the metrics
# Returns error for now because answers column is missing
# Error is misleading, fix dataset first and make it match the docs example dataset

result = evaluate(
    dataset_fix,
    metrics=[
        context_precision,
        faithfulness,
        answer_relevancy,
        context_recall,
    ],
)

result

result.to_pandas()


Evaluating:   0%|          | 0/28 [00:00<?, ?it/s]

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/chat/completions', 'files': None, 'json_data': {'messages': [{'content': 'Given a question, an answer, and sentences from the answer analyze the complexity of each sentence given under \'sentences\' and break down each sentence into one or more fully understandable statements while also ensuring no pronouns are used in each statement. Format the outputs in JSON.\n\nThe output should be a well-formatted JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output JSON schema:\n```\n{"type": "array", "items": {"$ref": "#/definitions/Statements"}, "definitions": {"Statements": {

Evaluating:   4%|▎         | 1/28 [00:01<00:29,  1.11s/it]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
DEBUG:openai._base_client:HTTP Request: POST https://api.openai.com/v1/chat/completions "200 OK"
DEBUG:openai._base_client:HTTP Request: POST https://api.openai.com/v1/chat/completions "200 OK"
DEBUG:openai._base_client:HTTP Request: POST https://api.openai.com/v1/chat/completions "200 OK"
DEBUG:openai._base_client:HTTP Request: POST https://api.openai.com/v1/chat/completions "200 OK"
DEBUG:openai._base_client:HTTP Request: POST https://api.openai.co

Evaluating:   7%|▋         | 2/28 [00:01<00:22,  1.14it/s]

...

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
DEBUG:openai._base_client:HTTP Request: POST https://api.openai.com/v1/chat/completions "200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
DEBUG:openai._base_client:HTTP Request: POST https://api.openai.com/v1/chat/completions "200 OK"


Evaluating:  96%|█████████▋| 27/28 [00:05<00:00,  4.69it/s]

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
DEBUG:openai._base_client:HTTP Request: POST https://api.openai.com/v1/chat/completions "200 OK"


Evaluating: 100%|██████████| 28/28 [00:06<00:00,  4.27it/s]


,question,ground_truth,answer,contexts,context_precision,faithfulness,answer_relevancy,context_recall
0,H,The Janelia Scientific Computing team supports...,T,[Overview of Janelia Scientific Computing\nThe...,1.0,NaN,0.754070,1.0
1,H,The Janelia Scientific Computing team utilizes...,T,[Overview of Janelia Scientific Computing\nThe...,1.0,NaN,0.754070,1.0
2,H,The Janelia Scientific Computing team utilizes...,T,[Overview of Janelia Scientific Computing\nThe...,1.0,NaN,0.754070,1.0
3,W,Some notable projects and contributions of the...,N,[Overview of Janelia Scientific Computing\nThe...,1.0,NaN,0.737152,1.0
4,H,The Janelia Scientific Computing team supports...,T,[Overview of Janelia Scientific Computing\nThe...,1.0,NaN,0.754070,1.0
5,W,The Janelia Scientific Computing team has impa...,N,[Overview of Janelia Scientific Computing\nThe...,1.0,NaN,0.737241,1.0
6,H,Neuroglancer is an open-source web-based tool ...,T,[Overview of Janelia Scientific Computing\nThe...,1.0,NaN,0.754070,1.0
